In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import time

# Baca data dari input.xlsx
df_input = pd.read_excel('input.xlsx')
results = []

# Siapkan browser
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.get("https://dms-siasn.bkn.go.id/")

# Tunggu login manual
input("Silakan login terlebih dahulu, lalu tekan ENTER untuk melanjutkan...")

# Looping tiap NIP
for index, row in df_input.iterrows():
    nip = str(row['nip'])
    print(f"Memproses NIP: {nip}")
    try:
        # Cari input field berdasarkan placeholder
        input_field = driver.find_element(By.XPATH, '//input[@placeholder="Masukan Nomor Induk Pegawai"]')
        input_field.clear()
        input_field.send_keys(nip)
        time.sleep(1)

        # Klik tombol "Load Data"
        load_button = driver.find_element(By.XPATH, '//button[contains(text(), "Load Data")]')
        load_button.click()

        time.sleep(3)  # Tunggu data load

        # Cek apakah tombol "Lanjut Verifikasi" ada
        try:
            lanjut_button = driver.find_element(By.XPATH, '//button[contains(text(), "Lanjut Verifikasi")]')
            lanjut_button.click()
            time.sleep(2)

            # Ambil nama pegawai - sesuaikan ini tergantung posisi elemen nama
            try:
                nama_element = driver.find_element(By.XPATH, '//div[contains(text(), "Nama")]/following-sibling::div')
                nama = nama_element.text.strip()
            except NoSuchElementException:
                nama = 'Nama tidak ditemukan'

        except NoSuchElementException:
            nama = 'Tidak ada tombol Lanjut Verifikasi'

        # Simpan hasil
        results.append({'nip': nip, 'nama': nama})

        # Tekan ENTER untuk lanjut ke berikutnya
        input("Tekan ENTER untuk memproses NIP berikutnya...")

    except Exception as e:
        print(f"Terjadi error pada NIP {nip}: {e}")
        results.append({'nip': nip, 'nama': 'Error saat proses'})

# Simpan ke report.xlsx
df_output = pd.DataFrame(results)
df_output.to_excel('report.xlsx', index=False)
print("Selesai. Hasil disimpan di report.xlsx")

# Tutup browser
driver.quit()


KeyboardInterrupt: 